# Trader Behavior Insights

This notebook is a step-by-step template to analyze how Bitcoin market sentiment (Fear/Greed) relates to Hyperliquid trader performance.

**Instructions:** upload the two CSV files into a `data/` directory and run the cells.



In [ ]:
# Imports and settings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
sns.set(style='whitegrid')
%matplotlib inline


In [ ]:
# Paths - update if needed
DATA_DIR = Path('data')
TRADER_CSV = DATA_DIR / 'hyperliquid_historical.csv'   # <- place your trader CSV here
SENTIMENT_CSV = DATA_DIR / 'fear_greed_index.csv'      # <- place your sentiment CSV here

print('Trader file:', TRADER_CSV)
print('Sentiment file:', SENTIMENT_CSV)


In [ ]:
# Load data (the parse_dates may need column name adjustment)
trader = pd.read_csv(TRADER_CSV, low_memory=False)
sent = pd.read_csv(SENTIMENT_CSV, low_memory=False)
print('Trader shape:', trader.shape)
print('Sentiment shape:', sent.shape)
trader.head()

In [ ]:
# Basic cleaning - normalize column names
trader.columns = [c.strip().lower().replace(' ', '_') for c in trader.columns]
trader.columns[:20]

In [ ]:
# Ensure time column parsed and create date
if 'time' in trader.columns:
    trader['time'] = pd.to_datetime(trader['time'], errors='coerce')
elif 'timestamp' in trader.columns:
    trader['time'] = pd.to_datetime(trader['timestamp'], errors='coerce')
else:
    print('No obvious time column found - please update this cell to use correct timestamp column.')
trader['date'] = trader['time'].dt.date
trader[['time','date']].head()

In [ ]:
# Convert numeric columns
for col in ['closedpnl','size','leverage','execution_price']:
    if col in trader.columns:
        trader[col] = pd.to_numeric(trader[col], errors='coerce')
trader[['closedpnl','size','leverage']].describe()

In [ ]:
# Clean sentiment dataset
sent.columns = [c.strip().lower().replace(' ', '_') for c in sent.columns]
# Assume sentiment has 'date' or 'date' column and a 'classification' column
if 'date' in sent.columns:
    sent['date'] = pd.to_datetime(sent['date'], errors='coerce').dt.date
elif 'datetime' in sent.columns:
    sent['date'] = pd.to_datetime(sent['datetime'], errors='coerce').dt.date
else:
    print('Check sentiment CSV column names.')
sent[['date']].head()

In [ ]:
# Aggregate trader metrics per date
agg = (trader.groupby('date')
       .agg(trades_count = ('account','size'),
            total_pnl = ('closedpnl','sum'),
            avg_pnl = ('closedpnl','mean'),
            median_pnl = ('closedpnl','median'))
       .reset_index())
# Win rate per date (separate calculation)
def win_rate_by_date(df):
    return df.groupby('date').apply(lambda g: (g['closedpnl']>0).sum()/len(g) if len(g)>0 else np.nan).reset_index(name='win_rate')
wr = win_rate_by_date(trader)
agg = agg.merge(wr, on='date', how='left')
agg.head()

In [ ]:
# Merge with sentiment
merged = agg.merge(sent[['date','classification']], on='date', how='left')
merged['classification'] = merged['classification'].fillna('Unknown')
merged.groupby('classification').agg({'total_pnl':['mean','median','std'],'trades_count':'mean','win_rate':'mean'})

In [ ]:
# Visuals: total PnL by sentiment
plt.figure(figsize=(8,5))
sns.boxplot(data=merged[merged['classification']!='Unknown'], x='classification', y='total_pnl')
plt.title('Daily total PnL by Market Sentiment (Fear vs Greed)')
plt.ylabel('Total PnL (USD or asset units)')
plt.show()

In [ ]:
# Example statistical test: Mann-Whitney U test for total_pnl between Fear and Greed
from scipy.stats import mannwhitneyu
fear = merged[merged['classification']=='Fear']['total_pnl'].dropna()
greed = merged[merged['classification']=='Greed']['total_pnl'].dropna()
if len(fear)>0 and len(greed)>0:
    stat, p = mannwhitneyu(fear, greed, alternative='two-sided')
    print('Mann-Whitney U stat=%.3f, p=%.3f' % (stat,p))
else:
    print('Not enough data for test')

## Extensions and suggestions

- Break down by `symbol` (e.g., BTC perpetuals vs others).
- Analyze leverage distribution across sentiments.
- Build a simple classifier to predict 'good day for traders' using sentiment + technical indicators.
- Compute per-account behavior: does some accounts systematically profit during Greed?



## Save results
You can save `merged` or aggregated tables as CSV for reporting:

```python
merged.to_csv('daily_agg_with_sentiment.csv', index=False)
```
